In [1]:
# Change to your dataset directory
import os
os.chdir('D:/学习/9444dataset/dataset_remade')

In [2]:

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from glob import glob
import fnmatch

In [3]:
# Collect paths to all images
imagePatches = glob('**/*.png', recursive=True)

In [4]:
len(imagePatches)

10384

In [5]:
# Separate image paths based on class
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)

In [6]:
total_num = 10384
count = 0
y = []
progress_increment = total_num * 0.05  # 5% of total
next_progress = progress_increment

for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)

    count += 1

    if count >= next_progress:
        print(f'Progress: {count/total_num*100:.2f}%')  # Prints the progress as a percentage
        next_progress += progress_increment


Progress: 5.01%
Progress: 10.01%
Progress: 15.00%
Progress: 20.00%
Progress: 25.00%
Progress: 30.01%
Progress: 35.01%
Progress: 40.00%
Progress: 45.00%
Progress: 50.00%
Progress: 55.01%
Progress: 60.01%
Progress: 65.00%
Progress: 70.00%
Progress: 75.00%
Progress: 80.01%
Progress: 85.01%
Progress: 90.00%
Progress: 95.00%


In [7]:
# Create dataframe
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y

In [8]:
# Split data into training and validation sets
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)

In [9]:
# Dataset class
class MyDataset(Dataset):
    def __init__(self, df_data,transform=None):
        super().__init__()
        self.df = df_data.values
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224,224))  # Resize images to 224x224
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [10]:
# Model parameters
num_epochs = 10
num_classes = 2
batch_size = 64
learning_rate = 0.002

In [11]:
# Define transforms
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])


In [12]:
# Create datasets and dataloaders
dataset_train = MyDataset(df_data=train, transform=trans_train)
dataset_valid = MyDataset(df_data=val,transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [13]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define the model
model = models.resnet50(pretrained=False)  # If you want to use the pretrained model, set pretrained=True

# Change the final layer to match the number of classes in the dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move the model to the GPU if available
model = model.to(device)

In [14]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [15]:
# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/130], Loss: 0.3619
Epoch [2/10], Step [100/130], Loss: 0.2594
Epoch [3/10], Step [100/130], Loss: 0.2332
Epoch [4/10], Step [100/130], Loss: 0.2043
Epoch [5/10], Step [100/130], Loss: 0.1304
Epoch [6/10], Step [100/130], Loss: 0.2571
Epoch [7/10], Step [100/130], Loss: 0.3580
Epoch [8/10], Step [100/130], Loss: 0.1608
Epoch [9/10], Step [100/130], Loss: 0.2326
Epoch [10/10], Step [100/130], Loss: 0.3961


In [16]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
confusion_matrix = torch.zeros(2, 2)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        for t, p in zip(labels.view(-1), predicted.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the test images: 91.76697159364468 %


In [17]:
# Save the model checkpoint
torch.save(model.state_dict(), 'resnet_model_10000.ckpt')